In [ ]:
#importing all the libraries

import os
import numpy as np
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob

In [ ]:
class SkinCancerDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.annotation = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotation)

    def __getitem__(self, index):
        img_id = self.annotation.iloc[index, 1]#ImageID on column 2
        img_name = os.path.join(self.root_dir, img_id + '.jpg')
        image = Image.open(img_name).convert('RGB')

        y_label = torch.tensor(int(self.annotation.iloc[index]['dx']))

        if self.transform:
            image = self.transform(image)

        return image, y_label


In [ ]:


meta = pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")

image_paths = glob.glob("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/*.jpg") + \
              glob.glob("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/*.jpg")

df_paths = pd.DataFrame({
    "path": image_paths,
})

# Extract image_id (remove .jpg)
df_paths["image_id"] = df_paths["path"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

df = df_paths.merge(meta, on="image_id")


# 1) Get unique lesion ids
lesions = df['lesion_id'].unique()

# 2) Split lesion IDs
train_lesions, test_lesions = train_test_split(
    lesions, test_size=0.20, random_state=42
)

train_lesions, val_lesions = train_test_split(
    train_lesions, test_size=0.125, random_state=42
)
# 0.125 of 80% ≈ 10% → final split = 70/10/20

train_df = df[df['lesion_id'].isin(train_lesions)].reset_index(drop=True)
val_df   = df[df['lesion_id'].isin(val_lesions)].reset_index(drop=True)
test_df  = df[df['lesion_id'].isin(test_lesions)].reset_index(drop=True)

print(len(train_df), len(val_df), len(test_df))

#img = Image.open(image_paths[0]).convert("RGB")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session